<a href="https://colab.research.google.com/github/mounika-b-25/Traffic-Collision-Analysis-and-Flow-Simulation-in-Toronto-Using-Cellular-Automata-Models/blob/main/Project_rough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


**Extract the zip files**

In [ ]:
import zipfile
import os

# List of zip files
zip_files = ["/content/archive (1).zip", "/content/archive (2).zip"]

# Extract each file
for zip_path in zip_files:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        extract_path = "/content/extracted_" + os.path.basename(zip_path).replace(".zip", "").replace(" ", "_")
        os.makedirs(extract_path, exist_ok=True)
        zip_ref.extractall(extract_path)
        print(f"Extracted: {zip_path} to {extract_path}")


Extracted: /content/archive (1).zip to /content/extracted_archive_(1)
Extracted: /content/archive (2).zip to /content/extracted_archive_(2)


**Import Libraries and Load the Data**

In [ ]:
import pandas as pd

# Load the datasets
traffic_df = pd.read_csv('/content/extracted_archive_(1)/Traffic_Collisions_Toronto_data.csv')
ksi_df = pd.read_csv('/content/extracted_archive_(2)/KSI.csv')


**Inspect the Data**

In [ ]:
# Traffic dataset
print("Traffic Collisions Dataset Info:")
print(traffic_df.shape)
print(traffic_df.columns)
print(traffic_df.head())

# KSI dataset
print("\nKSI Dataset Info:")
print(ksi_df.shape)
print(ksi_df.columns)
print(ksi_df.head())


Traffic Collisions Dataset Info:
(499538, 19)
Index(['X', 'Y', 'OBJECTID', 'EventUniqueId', 'OccurrenceDate', 'Month',
       'Day_of_Week', 'Year', 'Hour', 'Division', 'Atom', 'Neighbourhood',
       'Fatalities', 'Injury_Collisions', 'FTR_Collisions', 'PD_Collisions',
       'Longitude', 'Latitude', 'ObjectId2'],
      dtype='object')
     X    Y  OBJECTID   EventUniqueId          OccurrenceDate     Month  \
0  0.0  0.0         1     GO-20141001  2014/02/07 05:00:00+00  February   
1  0.0  0.0         2  GO-20141225593  2014/01/02 05:00:00+00   January   
2  0.0  0.0         3  GO-20141260499  2014/01/01 05:00:00+00   January   
3  0.0  0.0         4  GO-20141260663  2014/01/01 05:00:00+00   January   
4  0.0  0.0         5  GO-20141261162  2014/01/01 05:00:00+00   January   

  Day_of_Week  Year  Hour Division Atom Neighbourhood  Fatalities  \
0      Friday  2014    16      NSA  NSA           NSA           0   
1    Thursday  2014     3      NSA  NSA           NSA           0   
2  

- **Clean the KSI Dataset**  
 - Dropped unused index/object columns.  
 - Parsed DATE into datetime and renamed to Date.  
 - Standardized column names (Latitude, Longitude, Neighbourhood, Police_Division).  
 - Cleaned the Fatalities column similarly to traffic_df.

In [ ]:
# Drop unnecessary columns
traffic_df = traffic_df.drop(columns=['OBJECTID', 'ObjectId2'], errors='ignore')

# Convert 'OccurrenceDate' to datetime
traffic_df['OccurrenceDate'] = pd.to_datetime(traffic_df['OccurrenceDate'], errors='coerce')

# Rename key columns for consistency
traffic_df = traffic_df.rename(columns={
    'OccurrenceDate': 'Date',
    'Longitude': 'Longitude',
    'Latitude': 'Latitude',
    'Neighbourhood': 'Neighbourhood',
    'Division': 'Police_Division'
})

# Optional: Fill or flag missing values
traffic_df['Fatalities'] = pd.to_numeric(traffic_df['Fatalities'], errors='coerce').fillna(0)

# Display cleaned traffic_df
print("\nCleaned Traffic Dataset Sample:")
print(traffic_df.head())



Cleaned Traffic Dataset Sample:
     X    Y   EventUniqueId                      Date     Month Day_of_Week  \
0  0.0  0.0     GO-20141001 2014-02-07 05:00:00+00:00  February      Friday   
1  0.0  0.0  GO-20141225593 2014-01-02 05:00:00+00:00   January    Thursday   
2  0.0  0.0  GO-20141260499 2014-01-01 05:00:00+00:00   January   Wednesday   
3  0.0  0.0  GO-20141260663 2014-01-01 05:00:00+00:00   January   Wednesday   
4  0.0  0.0  GO-20141261162 2014-01-01 05:00:00+00:00   January   Wednesday   

   Year  Hour Police_Division Atom Neighbourhood  Fatalities  \
0  2014    16             NSA  NSA           NSA           0   
1  2014     3             NSA  NSA           NSA           0   
2  2014     2             NSA  NSA           NSA           0   
3  2014     3             NSA  NSA           NSA           0   
4  2014     5             NSA  NSA           NSA           0   

  Injury_Collisions FTR_Collisions PD_Collisions  Longitude  Latitude  
0                NO             NO 

- **Clean traffic_df**  
 - Column Names: The code now uses the Date column for datetime conversion (since your data contains Date instead of OccurrenceDate).  
 - Error Handling: If there were any non-date values in the Date column, they will be coerced to NaT (Not a Time).  
 - Data Cleaning: Placeholder values ('NSA') are replaced with NaN, and Fatalities are converted to numeric values.  
 - Duplicate Removal: Duplicates in the dataset are dropped.  
 - Extracting Month and Weekday: The Month and DayOfWeek columns are created for better analysis.

**Data Cleaning and Preparation for KSI Dataset**

In [ ]:
# Drop unnecessary columns
ksi_df = ksi_df.drop(columns=['INDEX_', 'ObjectId'], errors='ignore')

# Convert 'DATE' to datetime
ksi_df['DATE'] = pd.to_datetime(ksi_df['DATE'], errors='coerce')

# Rename key columns for consistency
ksi_df = ksi_df.rename(columns={
    'DATE': 'Date',
    'LONGITUDE': 'Longitude',
    'LATITUDE': 'Latitude',
    'NEIGHBOURHOOD': 'Neighbourhood',
    'POLICE_DIVISION': 'Police_Division',
    'FATAL_NO': 'Fatalities'
})

# Optional: Fill or convert fatality values
ksi_df['Fatalities'] = pd.to_numeric(ksi_df['Fatalities'], errors='coerce').fillna(0)

# Display cleaned ksi_df
print("\nCleaned KSI Dataset Sample:")
print(ksi_df.head())



Cleaned KSI Dataset Sample:
              X             Y  ACCNUM  YEAR                      Date  TIME  \
0 -8.844611e+06  5.412414e+06  892658  2006 2006-03-11 05:00:00+00:00   852   
1 -8.844611e+06  5.412414e+06  892658  2006 2006-03-11 05:00:00+00:00   852   
2 -8.816480e+06  5.434843e+06  892810  2006 2006-03-11 05:00:00+00:00   915   
3 -8.816480e+06  5.434843e+06  892810  2006 2006-03-11 05:00:00+00:00   915   
4 -8.822759e+06  5.424516e+06  892682  2006 2006-03-12 05:00:00+00:00   240   

   HOUR          STREET1           STREET2  OFFSET  ... EMERG_VEH PASSENGER  \
0     8       BLOOR ST W       DUNDAS ST W  <Null>  ...    <Null>    <Null>   
1     8       BLOOR ST W       DUNDAS ST W  <Null>  ...    <Null>    <Null>   
2     9  MORNINGSIDE AVE    SHEPPARD AVE E  <Null>  ...    <Null>    <Null>   
3     9  MORNINGSIDE AVE    SHEPPARD AVE E  <Null>  ...    <Null>    <Null>   
4     2   EGLINTON AVE E  COMMONWEALTH AVE  <Null>  ...    <Null>    <Null>   

  SPEEDING AG_DRIV  R

In [ ]:
# Check column names
print(traffic_df.columns)


Index(['X', 'Y', 'EventUniqueId', 'Date', 'Month', 'Day_of_Week', 'Year',
       'Hour', 'Police_Division', 'Atom', 'Neighbourhood', 'Fatalities',
       'Injury_Collisions', 'FTR_Collisions', 'PD_Collisions', 'Longitude',
       'Latitude'],
      dtype='object')


In [ ]:
# Rename key columns for consistency
traffic_df = traffic_df.rename(columns={
    'occurred_at': 'Date',
    'Longitude': 'Longitude',
    'Latitude': 'Latitude',
    'Neighbourhood': 'Neighbourhood',
    'Division': 'Police_Division'
})


In [ ]:
# Check if 'OccurrenceDate' exists before attempting to convert
if 'OccurrenceDate' in traffic_df.columns:
    traffic_df['OccurrenceDate'] = pd.to_datetime(traffic_df['OccurrenceDate'], errors='coerce')
else:
    print("'OccurrenceDate' column not found.")


'OccurrenceDate' column not found.


**Data Cleaning and Preparation for Traffic Dataset**

In [ ]:
import pandas as pd
import numpy as np

# Drop unnecessary columns
traffic_df = traffic_df.drop(columns=['OBJECTID', 'ObjectId2'], errors='ignore')

# Convert 'Date' to datetime
traffic_df['Date'] = pd.to_datetime(traffic_df['Date'], errors='coerce')

# Rename key columns for consistency
traffic_df = traffic_df.rename(columns={
    'Longitude': 'Longitude',
    'Latitude': 'Latitude',
    'Neighbourhood': 'Neighbourhood',
    'Division': 'Police_Division'
})

# Replace placeholder values with NaN (e.g., 'NSA' placeholder)
traffic_df.replace('NSA', np.nan, inplace=True)

# Convert Fatalities to numeric and fill missing with 0
traffic_df['Fatalities'] = pd.to_numeric(traffic_df['Fatalities'], errors='coerce').fillna(0)

# Drop duplicates
traffic_df.drop_duplicates(inplace=True)

# Optional: Extract month and weekday
traffic_df['Month'] = traffic_df['Date'].dt.month_name()
traffic_df['DayOfWeek'] = traffic_df['Date'].dt.day_name()

# View cleaned traffic dataset
print("\nCleaned Traffic Dataset Sample:")
print(traffic_df.head())



Cleaned Traffic Dataset Sample:
     X    Y   EventUniqueId                      Date     Month Day_of_Week  \
0  0.0  0.0     GO-20141001 2014-02-07 05:00:00+00:00  February      Friday   
1  0.0  0.0  GO-20141225593 2014-01-02 05:00:00+00:00   January    Thursday   
2  0.0  0.0  GO-20141260499 2014-01-01 05:00:00+00:00   January   Wednesday   
3  0.0  0.0  GO-20141260663 2014-01-01 05:00:00+00:00   January   Wednesday   
4  0.0  0.0  GO-20141261162 2014-01-01 05:00:00+00:00   January   Wednesday   

   Year  Hour Police_Division Atom Neighbourhood  Fatalities  \
0  2014    16             NaN  NaN           NaN           0   
1  2014     3             NaN  NaN           NaN           0   
2  2014     2             NaN  NaN           NaN           0   
3  2014     3             NaN  NaN           NaN           0   
4  2014     5             NaN  NaN           NaN           0   

  Injury_Collisions FTR_Collisions PD_Collisions  Longitude  Latitude  \
0                NO             NO

In [ ]:
# Check available columns to verify the exact column names
print("\nColumn Names in the KSI Dataset:")
print(ksi_df.columns)

# If the 'DATE' column is found with different casing, let's inspect and rename it
ksi_df.columns = ksi_df.columns.str.strip()  # Remove any leading/trailing spaces



Column Names in the KSI Dataset:
Index(['X', 'Y', 'ACCNUM', 'YEAR', 'Date', 'TIME', 'HOUR', 'STREET1',
       'STREET2', 'OFFSET', 'ROAD_CLASS', 'DISTRICT', 'WARDNUM', 'DIVISION',
       'Latitude', 'Longitude', 'LOCCOORD', 'ACCLOC', 'TRAFFCTL', 'VISIBILITY',
       'LIGHT', 'RDSFCOND', 'ACCLASS', 'IMPACTYPE', 'INVTYPE', 'INVAGE',
       'INJURY', 'Fatalities', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT',
       'DRIVCOND', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE', 'CYCACT',
       'CYCCOND', 'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK',
       'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER', 'SPEEDING', 'AG_DRIV',
       'REDLIGHT', 'ALCOHOL', 'DISABILITY', 'Police_Division', 'HOOD_ID',
       'Neighbourhood'],
      dtype='object')


In [ ]:
import pandas as pd
import numpy as np

# Drop unnecessary columns
ksi_df = ksi_df.drop(columns=['INDEX_', 'ObjectId'], errors='ignore')

# Convert 'Date' to datetime
ksi_df['Date'] = pd.to_datetime(ksi_df['Date'], errors='coerce')

# Rename key columns for consistency
ksi_df = ksi_df.rename(columns={
    'Date': 'Date',
    'Longitude': 'Longitude',
    'Latitude': 'Latitude',
    'Neighbourhood': 'Neighbourhood',
    'Police_Division': 'Police_Division',
    'Fatalities': 'Fatalities'
})

# Replace placeholder values with NaN
ksi_df.replace('<Null>', np.nan, inplace=True)

# Convert Fatalities to numeric and fill missing with 0
ksi_df['Fatalities'] = pd.to_numeric(ksi_df['Fatalities'], errors='coerce').fillna(0)

# Drop duplicates
ksi_df.drop_duplicates(inplace=True)

# Optional: Extract month and weekday
ksi_df['Month'] = ksi_df['Date'].dt.month_name()
ksi_df['DayOfWeek'] = ksi_df['Date'].dt.day_name()

# View cleaned KSI dataset
print("\nCleaned KSI Dataset Sample:")
print(ksi_df.head())



Cleaned KSI Dataset Sample:
              X             Y  ACCNUM  YEAR                      Date  TIME  \
0 -8.844611e+06  5.412414e+06  892658  2006 2006-03-11 05:00:00+00:00   852   
1 -8.844611e+06  5.412414e+06  892658  2006 2006-03-11 05:00:00+00:00   852   
2 -8.816480e+06  5.434843e+06  892810  2006 2006-03-11 05:00:00+00:00   915   
3 -8.816480e+06  5.434843e+06  892810  2006 2006-03-11 05:00:00+00:00   915   
4 -8.822759e+06  5.424516e+06  892682  2006 2006-03-12 05:00:00+00:00   240   

   HOUR          STREET1           STREET2 OFFSET  ... SPEEDING AG_DRIV  \
0     8       BLOOR ST W       DUNDAS ST W    NaN  ...      NaN     Yes   
1     8       BLOOR ST W       DUNDAS ST W    NaN  ...      NaN     Yes   
2     9  MORNINGSIDE AVE    SHEPPARD AVE E    NaN  ...      NaN     Yes   
3     9  MORNINGSIDE AVE    SHEPPARD AVE E    NaN  ...      NaN     Yes   
4     2   EGLINTON AVE E  COMMONWEALTH AVE    NaN  ...      NaN     NaN   

  REDLIGHT ALCOHOL  DISABILITY  Police_Divisi

**Checking Missing Values in Key Join Columns (Longitude, Latitude, Date, Police_Division)**

In [18]:
# Check for missing values in the key columns for both datasets
print("Missing values in Traffic dataset (join columns):")
print(traffic_df[['Longitude', 'Latitude', 'Date', 'Police_Division']].isnull().sum())

print("\nMissing values in KSI dataset (join columns):")
print(ksi_df[['Longitude', 'Latitude', 'Date', 'Police_Division']].isnull().sum())


Missing values in Traffic dataset (join columns):
Longitude              0
Latitude               0
Date                   0
Police_Division    69997
dtype: int64

Missing values in KSI dataset (join columns):
Longitude          0
Latitude           0
Date               0
Police_Division    0
dtype: int64


In [19]:
# Check unique values of join columns in both datasets
print("Unique 'Longitude' values in Traffic dataset:")
print(traffic_df['Longitude'].unique())

print("\nUnique 'Longitude' values in KSI dataset:")
print(ksi_df['Longitude'].unique())

# Repeat for other join columns (Latitude, Date, Police_Division)


Unique 'Longitude' values in Traffic dataset:
[  0.         -79.13288116 -79.13217549 ... -79.64239754 -79.63592147
 -79.63917958]

Unique 'Longitude' values in KSI dataset:
[-79.45249  -79.199786 -79.25619  ... -79.410084 -79.238926 -79.232021]


**Preprocessing: Rounding Coordinates, Formatting Dates, and Cleaning Police Division Names**

In [20]:
traffic_df['Police_Division'] = traffic_df['Police_Division'].fillna('Unknown')


In [21]:
traffic_df = traffic_df[traffic_df['Longitude'] != 0.0]


In [23]:
traffic_df['Longitude'] = traffic_df['Longitude'].round(4)
traffic_df['Latitude'] = traffic_df['Latitude'].round(4)
ksi_df['Longitude'] = ksi_df['Longitude'].round(4)
ksi_df['Latitude'] = ksi_df['Latitude'].round(4)


In [24]:
traffic_df['Date'] = pd.to_datetime(traffic_df['Date']).dt.date
ksi_df['Date'] = pd.to_datetime(ksi_df['Date']).dt.date


In [25]:
traffic_df['Police_Division'] = traffic_df['Police_Division'].astype(str).str.lower().str.strip()
ksi_df['Police_Division'] = ksi_df['Police_Division'].astype(str).str.lower().str.strip()


In [28]:
print(ksi_df.columns)


Index(['X', 'Y', 'ACCNUM', 'YEAR', 'Date', 'TIME', 'HOUR', 'STREET1',
       'STREET2', 'OFFSET', 'ROAD_CLASS', 'DISTRICT', 'WARDNUM', 'DIVISION',
       'Latitude', 'Longitude', 'LOCCOORD', 'ACCLOC', 'TRAFFCTL', 'VISIBILITY',
       'LIGHT', 'RDSFCOND', 'ACCLASS', 'IMPACTYPE', 'INVTYPE', 'INVAGE',
       'INJURY', 'Fatalities', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT',
       'DRIVCOND', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE', 'CYCACT',
       'CYCCOND', 'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK',
       'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER', 'SPEEDING', 'AG_DRIV',
       'REDLIGHT', 'ALCOHOL', 'DISABILITY', 'Police_Division', 'HOOD_ID',
       'Neighbourhood', 'Month', 'DayOfWeek'],
      dtype='object')


In [29]:
# Drop columns that are completely empty
final_df = final_df.dropna(axis=1, how='all')


In [30]:
for col in ['INJURY', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND']:
    final_df[col] = final_df[col].fillna('Unknown')


**Exploratory Analysis: Unique Non-Null Values in Fully and Partially Missing Columns**

In [33]:
# Fully missing columns — check unique non-null values
fully_empty_cols = [
    'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE', 'CYCACT', 'CYCCOND',
    'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'EMERG_VEH',
    'REDLIGHT', 'ALCOHOL', 'DISABILITY'
]

# Partially missing columns
partially_empty_cols = ['PEDESTRIAN', 'CYCLIST', 'PASSENGER', 'SPEEDING',
                        'ALCOHOL', 'DISABILITY', 'REDLIGHT', 'TRSN_CITY_VEH', 'EMERG_VEH',
                        'MOTORCYCLE', 'TRUCK']

# Get unique non-null values in each column
for col in fully_empty_cols + partially_empty_cols:
    print(f"Unique values in column '{col}':")
    print(final_df[col].dropna().unique())
    print("-" * 50)


Unique values in column 'PEDTYPE':
['Vehicle is going straight thru inter.while ped cross with ROW'
 'Vehicle turns left while ped crosses with ROW at inter.'
 'Vehicle is going straight thru inter.while ped cross without ROW'
 'Vehicle hits the pedestrian walking or running out from between parked vehicles at mid-block'
 'Pedestrian hit at mid-block' 'Pedestrian hit on sidewalk or shoulder'
 'Vehicle turns right while ped crosses without ROW at inter.'
 'Vehicle turns left while ped crosses without ROW at inter.'
 'Vehicle turns right while ped crosses with ROW at inter.'
 'Pedestrian hit a PXO/ped. Mid-block signal'
 'Vehicle is reversing and hits pedestrian'
 'Pedestrian hit at private driveway' 'Other / Undefined' 'Unknown'
 'Pedestrian involved in a collision with transit vehicle anywhere along roadway']
--------------------------------------------------
Unique values in column 'PEDACT':
['Crossing with right of way' 'Crossing without right of way'
 'Crossing, no Traffic Control' 

In [34]:
condition_cols = ['PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE', 'CYCACT', 'CYCCOND']
for col in condition_cols:
    final_df[col] = final_df[col].fillna('Not Involved')


In [35]:
binary_cols = ['ALCOHOL', 'DISABILITY', 'REDLIGHT', 'TRSN_CITY_VEH', 'EMERG_VEH',
               'PEDESTRIAN', 'CYCLIST', 'PASSENGER', 'SPEEDING']

for col in binary_cols:
    final_df[col] = final_df[col].fillna('No')


In [36]:
vehicle_cols = ['MOTORCYCLE', 'TRUCK']
for col in vehicle_cols:
    final_df[col] = final_df[col].fillna('No')


In [37]:
final_df.dtypes


,0
X,float64
Y,float64
EventUniqueId,object
Date,object
Month,object
...,...
Police_Division,object
HOOD_ID,int64
Neighbourhood,object
Month,object


In [38]:
binary_cols = ['ALCOHOL', 'DISABILITY', 'REDLIGHT', 'TRSN_CITY_VEH', 'EMERG_VEH',
               'PEDESTRIAN', 'CYCLIST', 'PASSENGER', 'SPEEDING', 'MOTORCYCLE', 'TRUCK']

for col in binary_cols:
    final_df[col] = final_df[col].map({'Yes': 1, 'No': 0})


**Column Removal: Dropping Empty, Zero-Only, and 'Not Involved'-Dominated Features**  
- **Columns with Missing Data (OFFSET):**  
 - If a column has no values (completely empty), it doesn't provide any meaningful information for your analysis or modeling. The column OFFSET in your case might be empty, and if it has no value, keeping it in the dataset could add unnecessary noise or complexity without contributing to your analysis.  
- ** Columns with All Zero Values (Fatalities, MOTORCYCLE, TRUCK, etc.):**  
 - If a column consists of only one value (e.g., all zeros), it doesn’t provide variability, which is essential for modeling or analysis. For example, if the MOTORCYCLE and TRUCK columns have only 'Yes' or zeros, it won't add much value because the column doesn't have any variation.  
 - In this case, if these columns are all filled with zeros or have a constant value across all rows, they won’t help in distinguishing data points and may just add redundancy to the analysis.  
- **Columns with Irrelevant Data (PEDTYPE, PEDACT, etc.):**  
 - If columns like PEDTYPE, PEDACT, and CYCLISTYPE are filled with 'Not Involved' or irrelevant data for your analysis, they may not contribute meaningful insights. Dropping these columns can help to focus on the more relevant variables and simplify the dataset, reducing noise and making it easier to analyze.  
 - For example, if most of the data is labeled 'Not Involved', there’s little to no variation in those columns, and they won’t help in differentiating or analyzing the dataset.  
- **Improving Model Performance:**  
 - In some cases, columns that don't add any value might negatively affect the performance of machine learning models by introducing unnecessary complexity and increasing the risk of overfitting.



In [46]:
# Define lists of columns to drop
empty_cols = ['OFFSET']
zero_cols = ['Fatalities', 'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'REDLIGHT']
uninvolved_cols = ['PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE']

# Combine all
columns_to_drop = empty_cols + zero_cols + uninvolved_cols

# Drop from the DataFrame
final_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Show what was dropped
print("Dropped columns:", columns_to_drop)


Dropped columns: ['OFFSET', 'Fatalities', 'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'REDLIGHT', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE']


- If the column contains missing data or has no values: It's pointless to keep empty columns.  
- If the column doesn't contribute meaningful information: Columns that have constant values or that do not vary across data points can be safely removed.
- If the column contains irrelevant or non-informative data: If it's filled with repeated, uninformative values like "Not Involved" or "Unknown", it's better to drop those columns to focus on more meaningful ones.



In [47]:
# Save the cleaned DataFrame to a new CSV file
final_df.to_csv('/content/drive/MyDrive/MRP/cleaned_data.csv', index=False)

# Confirm the file has been saved
print("Cleaned dataset saved as 'cleaned_data.csv'")


Cleaned dataset saved as 'cleaned_data.csv'
